# Backward Stability by Example

Copyright (C) 2024 Andreas Kloeckner

<details>
<summary>MIT License</summary>
Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in
all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
THE SOFTWARE.
</details>

In [1]:
import numpy as np
import numpy.linalg as la

Make an example of a very poorly conditioned matrix, called the [Vandermonde matrix](https://en.wikipedia.org/wiki/Vandermonde_matrix).

In [2]:
n = 20
nodes = np.linspace(0, 1, n)
A = nodes.reshape(-1, 1) ** np.arange(n)

Check the condition number:

In [3]:
#clear
la.cond(A, 2)

1.1386974885860262e+16

Set up a "true" solution, and a right-hand side for a linear system:

In [4]:
xtrue = np.random.randn(n)
b = A @ xtrue

Solve it computationally, call the solution `xcomp`:

In [7]:
#clear
xcomp = mylinsolve(A, b)

Compute the **relative forward error** (i.e. the error in the solution):

In [8]:
#clear
la.norm(xcomp - xtrue, 2)/la.norm(xtrue, 2)

6.045039485389126

Compute the

- **relative backward error**
- i.e. the relative error in the right-hand side
- also known as the **relative residual**.

In [9]:
#clear
la.norm(A@xcomp - b, 2)/la.norm(b, 2)

7.08631463572801e-11

### Input Perturbation

Now *intentionally* perturb the right-hand side and repeat the experiment

In [10]:
#clear
b_perturbed = b + np.random.randn(n) * 1e-14
xcomp_perturbed = la.solve(A, b_perturbed)

Find the relative forward error vs `xtrue`:

In [11]:
la.norm(xcomp_perturbed - xtrue, 2)/la.norm(xtrue, 2)

1.8575638755026467

Find the relative forward error vs `xcomp`:

In [12]:
la.norm(xcomp_perturbed - xcomp, 2)/la.norm(xtrue, 2)

4.189407948984551

And the relative backward error:

In [13]:
la.norm(A@xcomp_perturbed - b, 2)/la.norm(b, 2)

7.421496319997422e-15

### Can we take backward stability for granted?

Here's code for (mostly) straightforward Gauss-Jordan elimination to solve a linear system:

In [14]:
def mylinsolve(A, b):
    n = len(A)
    Ab = np.concatenate([A, b.reshape(-1, 1)], axis=1)
    
    # reverse the rows
    Ab = Ab[::-1]
    
    for col in range(n):
        Ab[col] /= Ab[col, col]
    
        for row in range(n):
            if col == row:
                continue
            Ab[row] -= Ab[col]*Ab[row, col]

    return Ab[:, -1]

Test that it solves linear systems:

In [15]:
Atest = np.random.randn(5, 5)
btest = np.random.randn(5)

xtest = la.solve(Atest, btest)

la.norm(mylinsolve(Atest, btest) - xtest, 2) / la.norm(xtest, 2)

6.097520505420507e-16

Now repeat the above experiments with `mylinsolve`.